In [3]:
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json
import pymongo
import pandas as pd
from pymongo import MongoClient as Connection

connection = Connection()
db = connection.recipe_db
input_data = db.rezepte
data = json.loads(json_util.dumps(input_data.find()))
norm_data = json_normalize(data)

ingredients = json_normalize(data, record_path='ingredients', meta='id', record_prefix='ingredients_', errors='ignore' )
ingredients = pd.DataFrame(ingredients)

#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

#schmeiss alle rezepte raus, die weniger als 4 zutaten besitzen
ingredients_unique = ingredients[ingredients.groupby('id').ingredients_id.transform(len) > 4]

#regex zum aufräumen der zutatennamen
ingredients_unique['ingredients_name']=ingredients_unique['ingredients_name'].str.replace(r"\(.*\)","")
ingredients_unique['ingredients_name']=ingredients_unique['ingredients_name'].str.replace(r"[-_.+/ ]*\d[-_.+ ]*","")
ingredients_unique['ingredients_name']=ingredients_unique['ingredients_name'].str.replace(r"\b(ml|g|sliced|dried|ground|, chopped|, drained)\b","")

#entferne spalten, die nicht benötigt werden
ingredients_unique = ingredients_unique.drop(ingredients_unique.columns[[0, 3, 4]], axis=1)




/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [12]:


#schmeiss alle rezepte raus, die weniger als 4 zutaten besitzen
ingredients_unique = ingredients[ingredients.groupby('id').ingredients_id.transform(len) > 4]

In [15]:
#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

In [13]:
ingredients_unique['ingredients_id'].nunique()

792

In [16]:
ingredients_unique['ingredients_id'].value_counts()

16421    588
4342     392
4397     339
6307     316
16406    247
2496     210
1526     198
16157    160
4572     117
4279     111
20551    109
1684     102
3717     100
16317     97
4432      92
0         92
5107      89
16238     82
6305      81
4405      78
16403     77
4786      77
4409      76
16407     74
4292      71
16396     66
16386     66
16385     63
4529      63
6379      63
        ... 
10896      6
16408      6
111        6
2940       6
20318      6
16389      6
20699      5
4315       5
20414      5
3823       5
16427      5
2825       5
1718       5
18845      5
3788       5
20466      5
12057      5
4589       5
3690       5
16382      5
20496      5
4244       5
2323       5
2351       5
2845       5
1637       5
10841      5
10863      5
2863       5
1624       5
Name: ingredients_id, Length: 258, dtype: int64

In [67]:
categories = json_normalize(data, record_path='categories', meta='id', record_prefix='categories_', errors='ignore' )
categories = pd.DataFrame(categories)

#kopiere nur werte die Vegetarian enthalten
#categories_veg = categories[categories.categories_name == 'Vegetarian']

#lösche alle doppelten kategorie IDs
#cat_unique = categories.drop_duplicates(['id'], keep='last')

#benenne spalte um
categories.rename(columns={'categories_name': 'vegetarian'}, inplace=True)
#benenne Vegetarian in 1 um
#categories = categories.replace(['Vegetarian'], [1]) 


cat_unique_veg['id'].nunique()
categories['id'].value_counts()

22314     16
25267     16
70312     15
21092     15
20350     14
35304     14
59467     14
53560     14
22494     14
17445     14
24182     14
34689     14
95860     14
47613     13
17522     13
22450     13
91679     13
105540    13
14088     13
71141     13
51025     13
84863     13
16259     13
25920     12
222801    12
13881     12
17127     12
18313     12
12991     12
13982     12
          ..
26972      3
13509      3
9174       3
14811      3
216190     3
12347      2
18996      2
20320      2
17351      2
14469      2
88278      2
19854      2
23783      2
90105      2
105039     2
24043      2
186378     2
73737      2
57124      2
230050     2
65300      2
22960      2
188690     2
64116      2
25528      2
21126      2
23081      2
13113      2
72616      2
220661     2
Name: id, Length: 988, dtype: int64

In [68]:
categories

,vegetarian,id
0,Vegetarian,43742
1,Italian,43742
2,Pasta,43742
3,World,43742
4,Main,43742
5,Dinner,43742
6,Gluten Free,14184
7,Vegan,14184
8,Dairy Free,14184
9,Vegetarian,14184


,vegetarian,id
3112,0,169278
3965,0,14522
3964,0,14522
3963,0,14522
3961,0,52934
3960,0,52934
3959,0,52934
3958,0,52934
3956,0,52934
3967,0,14440


In [69]:
import numpy as np
categories['vegetarian'] = np.where(categories['vegetarian']=='Vegetarian', 1, 0)

categories = categories.sort_values("vegetarian", ascending=False)
categories_veg = categories.drop_duplicates(["id"])

In [78]:
categories_veg['id'].value_counts()

90105     1
12991     1
25273     1
25272     1
8887      1
21174     1
25267     1
15025     1
17071     1
21166     1
21165     1
21160     1
41638     1
15013     1
230050    1
85881     1
15006     1
25075     1
25241     1
25239     1
150166    1
25237     1
146065    1
13914     1
82575     1
21134     1
21132     1
17033     1
23176     1
70330     1
         ..
38277     1
146824    1
11691     1
52232     1
228584    1
155074    1
87678     1
13043     1
247381    1
23988     1
73139     1
11698     1
11696     1
218542    1
11692     1
13907     1
19850     1
214048    1
69027     1
56737     1
11679     1
73975     1
11669     1
54675     1
19858     1
23953     1
9615      1
19854     1
191885    1
24578     1
Name: id, Length: 988, dtype: int64

In [75]:
ingredients_unique['id'].nunique()

986

In [62]:
asd = ingredients_unique.join(categories_veg.set_index(['id']), on=['id'])




In [79]:
categories_veg

,vegetarian,id
0,1,43742
4916,1,15359
1560,1,15084
1555,1,23421
4904,1,9111
1549,1,15176
4908,1,235259
1544,1,25156
4911,1,22960
4913,1,24291


In [89]:
categories_veg

,vegetarian,id
0,1,43742
4916,1,15359
1560,1,15084
1555,1,23421
4904,1,9111
1549,1,15176
4908,1,235259
1544,1,25156
4911,1,22960
4913,1,24291


In [92]:
ing_trans

,0,111,126,1416,1502,1507,1525,1526,1527,1536,...,20453,20466,20473,20482,20496,20551,20699,20726,22106,23274
id,,,,,,,,,,,,,,,,,,,,,
7025,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7034,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7915,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8626,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8691,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8847,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
ing_trans = pd.get_dummies(ingredients_unique['ingredients_id']).groupby(ingredients_unique['id']).apply(max)

In [90]:
#asd = ing_trans.join(categories_veg.set_index(['id']), on=['id'])


ads = ing_trans.merge(categories_veg, right_on='id', left_index=True)


In [91]:
ads

,0,111,126,1416,1502,1507,1525,1526,1527,1536,...,20473,20482,20496,20551,20699,20726,22106,23274,vegetarian,id
117,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,7025
1658,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,7034
284,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,7915
1233,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8626
5938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8652
1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8665
1924,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8691
5860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8694
1227,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8847
5922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8887


In [93]:
ads[ads.isnull().any(axis=1)]

,0,111,126,1416,1502,1507,1525,1526,1527,1536,...,20473,20482,20496,20551,20699,20726,22106,23274,vegetarian,id


In [86]:
ing_trans.head()

,0,111,126,1416,1502,1507,1525,1526,1527,1536,...,20453,20466,20473,20482,20496,20551,20699,20726,22106,23274
id,,,,,,,,,,,,,,,,,,,,,
7025,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7034,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7915,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8626,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ing_trans.rename(columns={'categories_name': 'vegetarian'}, inplace=True)

In [84]:
ing_trans['id'].nunique()

KeyError: 'id'

In [46]:
categories.head

<bound method NDFrame.head of       vegetarian      id
0              1   43742
1              0   43742
2              0   43742
3              0   43742
4              0   43742
5              0   43742
6              0   14184
7              0   14184
8              0   14184
9              1   14184
10             0   14184
11             0   14184
12             0  155074
13             0  155074
14             0  155074
15             0  155074
16             0  155074
17             0  155074
18             1   20669
19             0   20669
20             0   20669
21             0   20669
22             0  152937
23             0  152937
24             0  152937
25             0  152937
26             1  152937
27             0  152937
28             0  152937
29             0  152937
...          ...     ...
6195           0  214048
6196           0  214048
6197           0  214048
6198           0  214048
6199           0  198617
6200           0  198617
6201           0  19

In [60]:
print(test.loc[test['vegetarian'] == 0.0])

      vegetarian      id
6009           0   80677
4115           0  205651
5938           0    8652
6175           0  254940
6178           0  209691
6184           0  100690
6171           0  261124
3616           0  231616
6155           0  231619
3579           0   20293
3593           0  105039
6211           0  158833
3402           0   14224
6206           0  223042
6221           0  244605
6195           0  214048
6194           0   40573
3481           0  215093
6202           0  198617
3740           0   33160
5207           0  247381
5037           0  244779
5885           0   51283
5896           0   12720
5860           0    8694
5871           0   16700
5922           0    8887
4900           0  256014
4882           0  261823
1233           0    8626
1227           0    8847
1187           0   12009
251            0   21176
258            0   21694
225            0   26257
223            0   17227
217            0   21174
239            0   16239
235            0   68532


In [56]:
print(categories.loc[categories['vegetarian'] == 1.0])

      vegetarian      id
0              1   43742
4916           1   15359
1560           1   15084
1555           1   23421
4904           1    9111
1549           1   15176
4908           1  235259
1544           1   25156
4911           1   22960
4913           1   24291
1538           1   19770
1530           1   12991
1637           1   24626
4922           1  228337
4925           1   16085
1520           1   14180
4932           1  145672
1515           1   59967
4935           1   80974
1510           1   46028
4943           1   70101
4947           1   59853
4896           1   19388
1564           1   58246
4887           1   17033
1572           1   13996
1632           1   19584
4841           1   17351
1626           1   13524
4843           1  188690
...          ...     ...
313            1   20808
5846           1  254692
154            1   15006
601            1   25312
5775           1  132280
5934           1   72683
711            1   13896
6091           1   22304
